<a href="https://colab.research.google.com/github/IgorBarreto/IgorBarreto.github.io/blob/main/DSA/cap06/Notebooks/funcao_janela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carregue os dados desta meneira


```
!wget - q https://IgorBarreto.github.io/[caminho do arquivo]
```




### configura tamanho da fonte da saida

In [100]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)
!pip install pyspark --quiet

<IPython.core.display.Javascript object>

### Donload dos dados

In [101]:
# download dos dados

! wget -q https://IgorBarreto.github.io/DSA/cap06/Datasets/vendas.csv
! wget -q https://IgorBarreto.github.io/DSA/cap06/Datasets/2012Q1-capitalbikeshare-tripdata.zip
! wget -q https://IgorBarreto.github.io/DSA/cap06/Datasets/2012Q2-capitalbikeshare-tripdata.zip

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Cria spark session

In [102]:
import pyspark
import zipfile
from pyspark.sql import SparkSession
spark = SparkSession\
  .builder\
  .appName('Sql spark')\
  .getOrCreate()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Carrega dados

In [148]:
arq= ['/content/2012Q1-capitalbikeshare-tripdata.zip','/content/2012Q2-capitalbikeshare-tripdata.zip']
for a in arq:
  with zipfile.ZipFile(a) as zip_ref:
    zip_ref.extractall()

tabelas = {'vendas':'vendas.csv',
          'bicicletas_temp':'2012Q1-capitalbikeshare-tripdata.csv',
          'bicicletas_temp2':'2012Q2-capitalbikeshare-tripdata.csv'}
for k, v in tabelas.items():
  df=spark.read.csv(v,header=True,inferSchema=True,sep=",")
  df.createOrReplaceTempView(k)
spark.sql(
    """
SELECT
  Duration duracao_segundos,
  `Start date` data_inicio,
  `End date` data_fim,
  `Start station number` numero_estacao_inicio,
  `Start station` estacao_inicio,
  `End station number` numero_estacao_fim,
  `End station` estacao_fim,
  `Bike number` numero_bicicleta,
  `Member type` tipo_membro
FROM bicicletas_temp
    """).createOrReplaceTempView('bicicletas')

spark.sql(
    """
SELECT
  Duration duracao_segundos,
  `Start date` data_inicio,
  `End date` data_fim,
  `Start station number` numero_estacao_inicio,
  `Start station` estacao_inicio,
  `End station number` numero_estacao_fim,
  `End station` estacao_fim,
  `Bike number` numero_bicicleta,
  `Member type` tipo_membro
FROM bicicletas_temp2
    """).createOrReplaceTempView('bicicletas2')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
SQL="""select * from vendas"""
spark.sql(SQL).show()

In [ ]:
# Total de vendas
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) over (partition by ano_fiscal) total_venda_ano
from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

In [ ]:
# NÃO FAZ SENTIDO
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) total_venda,
  sum(valor_venda) over (partition by ano_fiscal) total_venda_ano
from vendas
group by valor_venda,ano_fiscal,nome_funcionario,valor_venda
order by ano_fiscal"""
spark.sql(SQL).show()

In [ ]:
#
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) over (partition by nome_funcionario) total_venda_ano
from vendas
order by nome_funcionario"""
spark.sql(SQL).show()

In [ ]:
# Total de vendas por ano, funcionario e geral
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) over (partition by ano_fiscal) total_venda_ano,
  sum(valor_venda) over () total_venda_total

from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

In [ ]:
# Numero de vendas por ano, por funcionario e
# número total de vendas em todos os oanos
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  count(*) over (partition by ano_fiscal,nome_funcionario) total_venda_ano,
  count(*) over () total_de_vendas

from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

In [ ]:
# Numero de vendas por ano, por funcionario e número total de vendas em todos os oanos usando subquery
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  count(*) over (partition by ano_fiscal,nome_funcionario) total_venda_ano,
  count(*) over () total_de_vendas

from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

### Bicicletas

In [ ]:
#  data_inicio
#  data_fim
#  numero_estacao_inicio
#  estacao_inicio
#  numero_estacao_fim
#  estacao_fim
#  numero_bicileta
#  tipo_membro
SQL="""select `data_inicio` from  bicicletas"""
spark.sql(SQL).show()

In [ ]:
# Duração total do aluguel das bikes (em horas)
SQL="""
select
  sum(duracao_segundos)/3600  duracao_horas
from bicicletas"""
spark.sql(SQL).show()

# Duração total do aluguel das bikes (em horas), ao longo do tempo
SQL="""
select
  duracao_segundos,
  sum(duracao_segundos/3600) over(order by data_inicio) duracao__total_horas
from bicicletas"""
spark.sql(SQL).show()

In [ ]:
# Duracao total do aluguel das bikes (em horas), ao longo do tempo, por estação
# de inicio do aluguel da bike, quando a data de inicio é menor '2012-01-08'

SQL="""
select
  row_number() over(order by estacao_inicio,data_inicio) numero_linha,
  estacao_inicio,
  duracao_segundos,
  sum(duracao_segundos/3600) over(partition by estacao_inicio order by data_inicio) duracao__total_horas
from bicicletas
where data_inicio <'2012-01-08'
order by numero_linha
"""
spark.sql(SQL).show(60)

In [ ]:
# Qual é a média de tempo (em horas) de aluguél de bikes da estação ininicio 310177
SQL="""
select
  estacao_inicio,
  duracao_segundos/3600,
  avg(duracao_segundos/3600) over(partition by estacao_inicio order by data_inicio) media_tempo_aluguel
from bicicletas
where numero_estacao_inicio = 31017
"""
spark.sql(SQL).show()

In [ ]:
# Retornar:
# Estação de início, data de início e duração de cada aluguel de bike em segundos
# Duração total de aluguel das bikes ao longo do tempo por estação de início
# Duração média do aluguel de bikes ao longo do tempo por estação de início
# Número de aluguéis de bikes por estação ao longo do tempo
# Somente os registros quando a data de início for inferior a '2012-01-08'
SQL="""
select
  estacao_inicio,
  data_inicio,
  duracao_segundos,
  sum(duracao_segundos) over(partition by estacao_inicio order by data_inicio) duracao_total,
  avg(duracao_segundos) over(partition by estacao_inicio order by data_inicio) duracao_media,
  count(*) over(partition by estacao_inicio order by data_inicio) qtde_aluguel
from bicicletas
where data_inicio < '2012-01-08'
"""
spark.sql(SQL).show()

In [ ]:
# Retornar:
# Estação de início, data de início de cada aluguel de bike e duração de cada aluguel em segundos
# Número de aluguéis de bikes (independente da estação) ao longo do tempo
# Somente os registros quando a data de início for inferior a '2012-01-08'

SQL="""
SELECT
  estacao_inicio,
  data_inicio,
  duracao_segundos,
  row_number() over(partition by estacao_inicio order by data_inicio) linha_estacao
FROM bicicletas
where data_inicio < '2012-01-08'
"""

spark.sql(SQL).show()

In [ ]:
# ROW_NUMBER
# Estacao_inicio, data_inicio,duracao_segundos do aluguel e número de aluguéis ao longo do tempo
# para a estação 31000
SQL="""
SELECT
  estacao_inicio,
  cast(data_inicio as date) data_inicio,
  duracao_segundos,
  row_number() over(partition by estacao_inicio order by cast(data_inicio as date)) rowNumber_numero_alugueis,
  dense_rank() over(partition by estacao_inicio order by cast(data_inicio as date)) drank_numero_alugueis,
  rank() over(partition by estacao_inicio order by cast(data_inicio as date)) rank_numero_alugueis

FROM bicicletas
where data_inicio < '2012-01-08' AND numero_estacao_inicio =31000
"""

spark.sql(SQL).show()

In [ ]:
#lag e lead
SQL="""
SELECT
  estacao_inicio,
  cast(data_inicio as date) data_inicio,
  duracao_segundos,
  LAG(duracao_segundos,1) over(partition by estacao_inicio order by cast(data_inicio as date)) registro_lag,
  LEAD(duracao_segundos,1) over(partition by estacao_inicio order by cast(data_inicio as date)) registro_lead
FROM bicicletas
where data_inicio < '2012-01-08' AND numero_estacao_inicio =31000
"""

spark.sql(SQL).show()

In [ ]:
#window alias
SQL="""
SELECT
  estacao_inicio,
  cast(data_inicio as date) data_inicio,
  duracao_segundos,
  NTILE(2) Over(partition by estacao_inicio order by cast(data_inicio as date)) n_group_2,
  NTILE(4) Over(partition by estacao_inicio order by cast(data_inicio as date)) n_group_4,
  NTILE(6) Over(partition by estacao_inicio order by cast(data_inicio as date)) n_group_6
FROM bicicletas
where data_inicio < '2012-01-08'
window ntile_window as (partition by estacao_inicio order by cast(data_inicio as date))
"""

spark.sql(SQL).show()

In [ ]:

SQL = """
SELECT
  data_inicio,
  DATE(data_inicio),
  TIMESTAMP(data_inicio),
  YEAR(data_inicio),
  MONTH(data_inicio),
  EXTRACT(MONTH FROM data_inicio),
  DAY(data_inicio)
FROM bicicletas
"""
spark.sql(SQL).show()

In [ ]:
SQL ="""
SELECT
  data_inicio,
  DATE_ADD(data_inicio, interval 1 MONTH) as data_inicio,
  duracao_segundos
FROM bicicletas
WHERE numero_estacao_inicio = 31000
"""
spark.sql(SQL).show()

In [107]:
# 1 - Qual a média de tempo (em segundos) de duração do aluguel de bike por tipo de membro?
SQL="""
select
  tipo_membro,
  avg(duracao_segundos)
from bicicletas2
group by tipo_membro
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+-----------+---------------------+
|tipo_membro|avg(duracao_segundos)|
+-----------+---------------------+
|    Unknown|                653.0|
|     Member|     697.663521690083|
|     Casual|   2499.6840492664815|
+-----------+---------------------+



In [109]:
# 2 - Qual a média de tempo (em segundos) de duração do aluguel de bike por tipo de membro
# e por estação fim (onde as bikes são entregues após o aluguel)?

SQL="""
select
  tipo_membro,
  estacao_fim,
  avg(duracao_segundos)
from bicicletas2
group by tipo_membro,estacao_fim
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+-----------+--------------------+---------------------+
|tipo_membro|         estacao_fim|avg(duracao_segundos)|
+-----------+--------------------+---------------------+
|     Member|Metro Center / 12...|    712.2872139407937|
|     Casual|14th St & New Yor...|   2598.3668763102723|
|     Member|      21st & M St NW|    594.0447653429603|
|     Member|Georgia Ave and F...|    817.2728785357737|
|     Member|Benning Branch Li...|               775.95|
|     Member|   20th & Crystal Dr|    954.6701030927835|
|     Member|19th St & Constit...|     1069.53216374269|
|     Casual|USDA / 12th & Ind...|    3392.481149012567|
|     Member|       1st & M St NE|    735.2394890899415|
|     Casual|Rosslyn Metro / W...|    2965.273076923077|
|     Member|Minnesota Ave Met...|    1140.857142857143|
|     Casual|Rhode Island & Co...|   1840.1424332344213|
|     Member|S Glebe & Potomac...|    603.4880952380952|
|     Member|       4th & M St SW|    830.6654330125833|
|     Casual|      10th & U St 

In [133]:
# 3 - Qual hora do dia (independente do mês) a bike de número W01182 teve o maior número
# de aluguéis considerando a data de início?
SQL="""
select
  DATE_FORMAT(data_inicio,'HH') hora,
  count(duracao_segundos) contagem
from bicicletas2
where numero_bicicleta = 'W01182'
GROUP BY DATE_FORMAT(data_inicio,'HH')
ORDER BY contagem DESC
"""

spark.sql(SQL).show(25)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+----+--------+
|hora|contagem|
+----+--------+
|  17|      43|
|  18|      41|
|  16|      36|
|  08|      34|
|  15|      33|
|  19|      33|
|  13|      29|
|  20|      26|
|  12|      23|
|  14|      22|
|  07|      21|
|  11|      21|
|  09|      17|
|  21|      17|
|  10|      16|
|  22|      15|
|  00|       7|
|  23|       6|
|  06|       6|
|  02|       4|
|  01|       1|
|  05|       1|
+----+--------+



In [135]:
# 4 - Qual a média de tempo (em segundos) de duração do aluguel de bike por tipo de membro
# e por estação fim (onde as bikes são entregues após o aluguel) ao longo do tempo?
SQL="""
select
  tipo_membro,
  estacao_fim,
  avg(duracao_segundos) over(partition by tipo_membro order by data_inicio) media
from bicicletas2

"""

spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+-----------+--------------------+-----------------+
|tipo_membro|         estacao_fim|            media|
+-----------+--------------------+-----------------+
|     Member|       7th & T St NW|            475.0|
|     Member|16th & Harvard St NW|            810.0|
|     Member|Park Rd & Holmead...|701.6666666666666|
|     Member|       7th & T St NW|            644.0|
|     Member|Lynn & 19th St North|            586.8|
|     Member|  9th & Upshur St NW|781.3333333333334|
|     Member|      15th & P St NW|706.7142857142857|
|     Member|25th St & Pennsyl...|          682.875|
|     Member|L'Enfant Plaza / ...|708.4444444444445|
|     Member|Massachusetts Ave...|            747.3|
|     Member|      21st & I St NW|723.9090909090909|
|     Member|Massachusetts Ave...|750.6666666666666|
|     Member|14th & Rhode Isla...|772.5384615384615|
|     Member|14th & Rhode Isla...|793.0714285714286|
|     Member|Massachusetts Ave...|809.4666666666667|
|     Member|New York Ave & 15...|         786

In [152]:
# 5 - Qual o número de aluguéis da bike de número W01182 ao longo do tempo considerando
# a data de início?
SQL="""
SELECT
  CAST(data_inicio AS date) data_inicio,
  count(*) over(partition by estacao_inicio order by  CAST(data_inicio AS date)) contagem
FROM bicicletas2
where numero_bicicleta='W01182'
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+-----------+--------+
|data_inicio|contagem|
+-----------+--------+
| 2012-04-02|       1|
| 2012-04-10|       2|
| 2012-05-19|       1|
| 2012-04-15|       1|
| 2012-04-18|       2|
| 2012-04-28|       3|
| 2012-04-11|       1|
| 2012-04-01|       1|
| 2012-05-01|       2|
| 2012-06-13|       3|
| 2012-05-07|       1|
| 2012-04-12|       1|
| 2012-06-14|       2|
| 2012-04-06|       1|
| 2012-04-20|       2|
| 2012-05-24|       3|
| 2012-06-10|       4|
| 2012-06-22|       5|
| 2012-05-23|       1|
| 2012-06-13|       1|
+-----------+--------+
only showing top 20 rows



In [158]:
# 6 - Retornar:
#     Estação fim, data fim de cada aluguel de bike e duração de cada aluguel em segundos
#     Número de aluguéis de bikes (independente da estação) ao longo do tempo
#     Somente os registros quando a data fim foi no mês de Abril
SQL="""
SELECT
  estacao_fim,data_fim,duracao_segundos,
  count(*) over(order by data_fim) numero
FROM bicicletas2
where MONTH(data_fim) = 04
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+--------------------+-------------------+----------------+------+
|         estacao_fim|           data_fim|duracao_segundos|numero|
+--------------------+-------------------+----------------+------+
|      21st & M St NW|2012-04-01 00:04:26|             192|     1|
|Harvard St & Adam...|2012-04-01 00:06:57|             163|     2|
|Massachusetts Ave...|2012-04-01 00:09:45|             187|     3|
|Rhode Island & Co...|2012-04-01 00:10:26|             408|     4|
|20th St & Florida...|2012-04-01 00:11:23|             188|     5|
|25th St & Pennsyl...|2012-04-01 00:13:15|             474|     6|
|Rhode Island & Co...|2012-04-01 00:14:25|             492|     7|
|New Hampshire Ave...|2012-04-01 00:15:35|             260|     8|
|New Hampshire Ave...|2012-04-01 00:15:43|             504|     9|
|Harvard St & Adam...|2012-04-01 00:16:11|             169|    10|
|       3rd & D St SE|2012-04-01 00:17:00|             446|    11|
|      15th & P St NW|2012-04-01 00:17:28|             843|   

In [169]:
# 7 - Retornar:
#     Estação fim, data fim e duração em segundos do aluguel
#     A data fim deve ser retornada no formato: 01/January/2012 00:00:00
#     Queremos a ordem (classificação ou ranking) dos dias de aluguel ao longo dotempo
#     Retornar os dados para os aluguéis entre 7 e 11 da manhã
SQL="""
select
  estacao_fim,
  date_format(data_fim,"dd-MM-yyyy HH:mm:ss") data_fim,
  duracao_segundos,
  dense_rank() over (partition by estacao_fim order by cast(data_fim as date)) ranking
  from bicicletas
  where extract(hour from data_fim) between 07 and 11
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+-------------------+-------------------+----------------+-------+
|        estacao_fim|           data_fim|duracao_segundos|ranking|
+-------------------+-------------------+----------------+-------+
|10th & Monroe St NE|01-01-2012 11:01:02|             619|      1|
|10th & Monroe St NE|01-01-2012 11:37:46|            1956|      1|
|10th & Monroe St NE|01-01-2012 11:37:22|            1899|      1|
|10th & Monroe St NE|07-01-2012 09:50:03|            1096|      2|
|10th & Monroe St NE|10-01-2012 08:13:37|             862|      3|
|10th & Monroe St NE|14-01-2012 08:17:21|             784|      4|
|10th & Monroe St NE|14-01-2012 10:07:59|             761|      4|
|10th & Monroe St NE|16-01-2012 10:21:08|             756|      5|
|10th & Monroe St NE|21-01-2012 08:39:23|             891|      6|
|10th & Monroe St NE|22-01-2012 11:43:02|           22971|      7|
|10th & Monroe St NE|28-01-2012 08:32:06|             808|      8|
|10th & Monroe St NE|28-01-2012 10:17:52|             749|    

In [181]:
# 8 - Qual a diferença da duração do aluguel de bikesao longo do tempo, de um registro para
# outro, considerando data de início do aluguel e estação de início?
# A data de início deve ser retornada no
# formato: Sat/Jan/12 00:00:00 (Sat = Dia da semana abreviado e Jan igual mês abreviado).
# Retornar os dados para os aluguéis entre 01 e 03 da manhã

SQL="""
SELECT
    date_format(data_fim,"dd-MMM-yyyy HH:mm:ss") data_fim,
    duracao_segundos - LAG(duracao_segundos,1) over (partition by estacao_inicio order by cast(data_inicio as date)) as diff
FROM bicicletas2
 where extract(hour from data_fim) between 01 and 03
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+--------------------+-----+
|            data_fim| diff|
+--------------------+-----+
|05-Apr-2012 02:45:10| NULL|
|12-Apr-2012 02:23:22|  337|
|14-Apr-2012 02:16:33| -916|
|21-Apr-2012 02:04:46| 4660|
|28-Apr-2012 02:31:26|-4571|
|03-May-2012 02:31:48| -100|
|05-May-2012 03:44:55| 8431|
|25-May-2012 03:04:30| 2161|
|03-Jun-2012 01:35:20|-5812|
|24-Jun-2012 02:20:04|-4149|
|24-Jun-2012 02:20:06|  -10|
|01-Apr-2012 01:01:54| NULL|
|01-Apr-2012 01:30:33|  165|
|01-Apr-2012 01:43:20|  162|
|01-Apr-2012 01:46:52| -177|
|01-Apr-2012 02:20:15| 1280|
|01-Apr-2012 02:09:09|-1356|
|01-Apr-2012 02:18:11|  524|
|01-Apr-2012 02:21:03| -506|
|01-Apr-2012 02:53:20|  190|
+--------------------+-----+
only showing top 20 rows



In [186]:
# 9 - Retornar:
#     Estação fim, data fim e duração em segundos do aluguel
#     A data fim deve ser retornada no formato: 01/January/2012 00:00:00# Queremos os
#     registros divididos em 4 grupos ao longo do tempo por partição
#     Retornar os dados para os aluguéis entre 8 e 10 da manhã
#     Qual critério usado pela função NTILE para dividir os grupos?
SQL="""
SELECT
    estacao_fim,
    date_format(data_fim,"dd/MMMM/yyyy HH:mm:ss") data_fim,
    duracao_segundos,
    NTILE(4) over (partition by estacao_fim order by cast(data_fim as date)) as ranking
FROM bicicletas2
 where extract(hour from data_fim) between 08 and 11
"""
spark.sql(SQL).show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+-------------------+--------------------+----------------+-------+
|        estacao_fim|            data_fim|duracao_segundos|ranking|
+-------------------+--------------------+----------------+-------+
|10th & Monroe St NE|01/April/2012 11:...|             902|      1|
|10th & Monroe St NE|01/April/2012 11:...|             848|      1|
|10th & Monroe St NE|01/April/2012 11:...|             843|      1|
|10th & Monroe St NE|01/April/2012 11:...|             869|      1|
|10th & Monroe St NE|03/April/2012 10:...|             449|      1|
|10th & Monroe St NE|04/April/2012 09:...|             758|      1|
|10th & Monroe St NE|06/April/2012 11:...|            1542|      1|
|10th & Monroe St NE|09/April/2012 08:...|             497|      1|
|10th & Monroe St NE|11/April/2012 09:...|             856|      1|
|10th & Monroe St NE|13/April/2012 10:...|             566|      1|
|10th & Monroe St NE|14/April/2012 10:...|            1436|      1|
|10th & Monroe St NE|14/April/2012 10:...|      

In [192]:
# 10- Quais estações tiveram mais de 35 horas de duração total do aluguel de bike ao longo do
# tempo considerando a data fim e estação fim?
#     Retorne os dados entre os dias '2012-04-01' e '2012-04-02'
#     Dica: Use função window e subquery
SQL ="""
select * from (
SELECT
  estacao_fim, CAST(data_fim as date) data_fim,
  sum(duracao_segundos/3600) over (partition by estacao_fim order by cast(data_fim as date)) tempo_total_horas
FROM bicicletas2
where data_fim  between '2012-04-01' and '2012-04-02')
where tempo_total_horas > 35
"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+--------------------+----------+-----------------+
|         estacao_fim|  data_fim|tempo_total_horas|
+--------------------+----------+-----------------+
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & Constit...|2012-04-01|72.27388888888892|
|10th St & C